In [10]:
import utils.sql as sql_tools
import pandas as pd

In [48]:
cnx, cursor = sql_tools.create_session_cursor()

query = '''
SELECT od.order_id, od.food_id, food_name, food_type, quantity, price, order_time from 
order_details od inner join food inner join `order`
ON od.food_id = food.food_id AND od.order_id = `order`.order_id
WHERE od.order_id in (
	select order_id from `order` where restaurant_id = 2
);
'''

result = sql_tools.execute_fetchall(cursor, query, tuple())
result = pd.DataFrame(result)

result.columns = ['ORDER_ID', 'FOOD_ID', 'FOOD_NAME', 'FOOD_TYPE', 'QUANTITY', 'PRICE', 'ORDER_TIME']
result = result.astype({'QUANTITY': int})
len(result)
result.head()
# result.dtypes

,ORDER_ID,FOOD_ID,FOOD_NAME,FOOD_TYPE,QUANTITY,PRICE,ORDER_TIME
0,601,11,葱香烤鸡,荤菜,1,35.0,2023-03-10 17:18:29
1,601,13,卤制烤鸭,荤菜,1,45.0,2023-03-10 17:18:29
2,601,15,盐菜扣肉,荤菜,1,38.0,2023-03-10 17:18:29
3,601,16,鲜花椒鲈鱼,荤菜,1,56.0,2023-03-10 17:18:29
4,601,17,香锅卷心菜,素菜,1,25.0,2023-03-10 17:18:29


In [12]:
def _process_overturn_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order, then:
        1. Calculate the price * quantity
        2. Group by FOOD_TYPE
        3. Group by date (with viewmode)
    '''
    # step 1
    df['Money'] = df['QUANTITY'] * df['PRICE']
    # step 2
    ans = pd.DataFrame()
    df1 = df.groupby('FOOD_TYPE')
    for food_type, sub_df in df1:
        # step 3
        date_grouped = sub_df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode)).sum(numeric_only=True)
        ans[food_type] = date_grouped['Money']
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    return ans
_process_overturn_analysis(result, 'D')

,汤,素菜,荤菜
2023-03-01,1056.0,2453.0,8144.0
2023-03-02,1056.0,2453.0,7375.0
2023-03-03,864.0,1871.0,5507.0
2023-03-04,864.0,1933.0,6535.0
2023-03-05,928.0,2081.0,7193.0
2023-03-06,1120.0,2246.0,7947.0
2023-03-07,608.0,1642.0,5024.0
2023-03-08,704.0,1633.0,5318.0
2023-03-09,608.0,1881.0,5639.0
2023-03-10,960.0,2106.0,7471.0


In [13]:
def _process_overturn_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order, then:
        1. Calculate the price * quantity
        2. Group by date (with viewmode)
    '''
    # step 1
    df['Money'] = df['QUANTITY'] * df['PRICE']
    # step 2
    ans = df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode))['Money'].sum(numeric_only=True)
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    return ans
_process_overturn_analysis(result, 'D')

2023-03-01    11653.0
2023-03-02    10884.0
2023-03-03     8242.0
2023-03-04     9332.0
2023-03-05    10202.0
2023-03-06    11313.0
2023-03-07     7274.0
2023-03-08     7655.0
2023-03-09     8128.0
2023-03-10    10537.0
2023-03-11     7158.0
2023-03-12     9008.0
2023-03-13     8392.0
2023-03-14     8362.0
2023-03-15    10936.0
Name: Money, dtype: float64

In [14]:
import datetime
def _get_range_turnover(df, start_date, end_date):
    df['Money'] = df['QUANTITY'] * df['PRICE']
    mask = (df['ORDER_TIME'] > start_date) & (df['ORDER_TIME'] < end_date)
    return df.loc[mask]['Money'].sum()
_get_range_turnover(result, datetime.datetime(year=2023, month=3, day=15) - datetime.timedelta(days=7),
                    datetime.datetime(year=2023, month=3, day=15))

59240.0

In [15]:
def _get_range_orders(df, start_date, end_date):
    mask = (df['ORDER_TIME'] > start_date) & (df['ORDER_TIME'] < end_date)
    df = df.drop_duplicates(subset=['ORDER_ID'])
    return df.loc[mask]['ORDER_ID'].count()
_get_range_orders(result, datetime.datetime(year=2023, month=3, day=15) - datetime.timedelta(days=7),
                    datetime.datetime(year=2023, month=3, day=15))

254

In [16]:
def _get_range_dishes(df, start_date, end_date):
    mask = (df['ORDER_TIME'] > start_date) & (df['ORDER_TIME'] < end_date)
    return df.loc[mask]['ORDER_ID'].count()
_get_range_dishes(result, datetime.datetime(year=2023, month=3, day=15) - datetime.timedelta(days=7),
                    datetime.datetime(year=2023, month=3, day=15))

1586

In [21]:
def _process_dish_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order,
        Then group by food name and date respectively.
    '''
    ans = pd.DataFrame()
    df1 = df.groupby('FOOD_NAME')
    for food_name, sub_df in df1:
        date_grouped = sub_df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode)).sum(numeric_only=True)
        ans[food_name] = date_grouped['QUANTITY']
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    print(list(ans.columns[:2]))
    return ans.sum()
_process_dish_analysis(result)#[['剁椒鱼头', '小炒黄牛肉']]

['卤制烤鸭', '油淋菜心']


卤制烤鸭      363
油淋菜心      542
盐菜扣肉      232
石锅鸡汤豆腐    407
粉丝裹虾      471
葱香烤鸡      307
酱爆石锅蛙     418
香锅卷心菜     417
香锅花菜      327
鲜花椒鲈鱼     241
dtype: int64

In [33]:
result

,ORDER_ID,FOOD_ID,FOOD_NAME,FOOD_TYPE,QUANTITY,PRICE,ORDER_TIME,Money
0,601,11,葱香烤鸡,荤菜,1,35.0,2023-03-10 17:18:29,35.0
1,601,13,卤制烤鸭,荤菜,1,45.0,2023-03-10 17:18:29,45.0
2,601,15,盐菜扣肉,荤菜,1,38.0,2023-03-10 17:18:29,38.0
3,601,16,鲜花椒鲈鱼,荤菜,1,56.0,2023-03-10 17:18:29,56.0
4,601,17,香锅卷心菜,素菜,1,25.0,2023-03-10 17:18:29,25.0
...,...,...,...,...,...,...,...,...
3720,1200,12,酱爆石锅蛙,荤菜,1,58.0,2023-03-08 17:58:25,58.0
3721,1200,14,粉丝裹虾,荤菜,1,49.0,2023-03-08 17:58:25,49.0
3722,1200,17,香锅卷心菜,素菜,1,25.0,2023-03-08 17:58:25,25.0
3723,1200,18,油淋菜心,素菜,1,18.0,2023-03-08 17:58:25,18.0


In [64]:
cnx, cursor = sql_tools.create_session_cursor()

query = '''
SELECT * from food;
'''

food = sql_tools.execute_fetchall(cursor, query, tuple())
food = pd.DataFrame(food)[[0, 3]]
food.index = food[0]
food.columns = ['FOOD_ID', 'FOOD_NAME']
food

,FOOD_ID,FOOD_NAME
0,,
1,1,辣椒炒肉
2,2,小炒黄牛肉
3,3,剁椒鱼头
4,4,口味鸡
5,5,烟笋炒腊肉
6,6,有机花菜
7,7,煎茄子
8,8,玉米淮山
9,9,茶树菇土鸡汤


In [70]:
def _get_best_selling_dish(orderfood, food, best_n=3):
    ''' Get the `best_n` selling dishes. 
        Input: 
    '''
    best_n = min(best_n, 3)
    retval = pd.DataFrame(
        orderfood.groupby('FOOD_ID')
        .count()
        .sort_values(by=['ORDER_ID'], ascending=False)
        .head(best_n)['ORDER_ID']
    )
    retval = retval.rename(columns={'ORDER_ID': 'SELLING'})
    food_name = food.loc[retval.index, 'FOOD_NAME']
    retval['FOOD_NAME'] = food_name

    return retval
 
_get_best_selling_dish(result, food)  

,SELLING,FOOD_NAME
FOOD_ID,,
18,542,油淋菜心
14,471,粉丝裹虾
12,418,酱爆石锅蛙
